# LangChain 1.0 Middleware: Advanced Agent Control

## Overview
**Middleware** is LangChain 1.0's killer feature for **context engineering** - the practice of getting the right information to your model at the right time.

## What is Middleware?
Middleware provides hooks into your agent's execution lifecycle, allowing you to:
- **Control what happens before/after** LLM calls
- **Intercept and modify** tool executions
- **Add guardrails** without cluttering your main code
- **Manage context** to prevent pollution
- **Enforce limits** to control costs

## Why Middleware Matters
Think of middleware as **clean room protocols** for your agent:
- 🧼 **Context quarantine**: Keep information separate
- 🛡️ **Safety guardrails**: Screen inputs and outputs
- 📊 **Observability**: Log and monitor behavior
- 💰 **Cost control**: Limit expensive operations
- 👤 **Human oversight**: Pause for approval

## What You'll Learn
1. Built-in middleware classes
2. Creating custom middleware with decorators
3. Building class-based middleware
4. Combining multiple middleware layers
5. Real-world use cases and patterns

## Setup: Install and Import

In [1]:
import os
import logging
from typing import Any, Callable
from dotenv import load_dotenv

# LangChain 1.0 imports
from langchain.agents import create_agent
from langchain.agents.middleware import (
    ModelCallLimitMiddleware,
    ToolCallLimitMiddleware,
    SummarizationMiddleware,
    HumanInTheLoopMiddleware,
    TodoListMiddleware,
    AgentMiddleware,
    AgentState,
    before_model,
    after_model,
    before_agent,
    after_agent,
)
from langchain.tools.tool_node import ToolCallRequest
from langchain.messages import ToolMessage, AIMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools import tool
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.runtime import Runtime

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Load environment variables
load_dotenv()

print("✓ All libraries imported successfully")
print("✓ Ready to explore middleware!")

/Users/laurafunderburk/Documents/GitHub/Building-Natural-Language-Pipelines/ch2/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✓ All libraries imported successfully
✓ Ready to explore middleware!


## Part 1: Built-in Middleware Classes

LangChain 1.0 provides several ready-to-use middleware classes.

### 1.1 Model Call Limit Middleware

Prevents runaway costs by limiting how many times the model can be called.

In [2]:
# Create a simple search tool
search_tool = TavilySearchResults(max_results=2)

# Limit model calls
model_limiter = ModelCallLimitMiddleware(
    run_limit=3,  # Max 3 LLM calls per single invocation
    thread_limit=10,  # Max 10 calls across all runs in a thread
    exit_behavior="end"  # Gracefully end (vs "error" to raise exception)
)

# Create agent with middleware
limited_agent = create_agent(
    model="gpt-4o-mini",
    tools=[search_tool],
    middleware=[model_limiter],
    system_prompt="You are a research assistant. Be concise."
)

print("✓ Agent with model call limits created")
print("  - Max 3 calls per run")
print("  - Max 10 calls per thread")

✓ Agent with model call limits created
  - Max 3 calls per run
  - Max 10 calls per thread


/var/folders/k7/8m84jpf52x7gvx628v82h2p80000gn/T/ipykernel_11169/391585432.py:2: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  search_tool = TavilySearchResults(max_results=2)


In [3]:
# Test the limited agent
result = limited_agent.invoke({
    "messages": [{"role": "user", "content": "What are the latest AI trends?"}]
})

print("\nResponse:", result["messages"][-1].content[:200] + "...")
print(f"\n✓ Agent stayed within limits!")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Response: Here are the latest trends in AI for 2024:

1. **Enhanced Performance Metrics**: AI is excelling in benchmarks related to image classification and language understanding, although it still struggles w...

✓ Agent stayed within limits!


### 1.2 Tool Call Limit Middleware

Limits how many times a specific tool can be called.

In [4]:
# Create a custom tool
@tool
def expensive_api(query: str) -> str:
    """An expensive API call that we want to limit."""
    logger.info(f"💰 Expensive API called with: {query}")
    return f"Expensive result for: {query}"

# Limit tool calls
tool_limiter = ToolCallLimitMiddleware(
    tool_name="expensive_api",
    run_limit=2,  # Max 2 calls per invocation
    thread_limit=5  # Max 5 calls per thread
)

tool_limited_agent = create_agent(
    model="gpt-4o-mini",
    tools=[expensive_api],
    middleware=[tool_limiter],
    system_prompt="You can use the expensive API, but use it wisely."
)

print("✓ Agent with tool limits created")
print("  - expensive_api limited to 2 calls per run")

✓ Agent with tool limits created
  - expensive_api limited to 2 calls per run


### 1.3 Summarization Middleware

Automatically summarizes conversation history when it gets too long.

In [5]:
# Create summarization middleware
summarizer = SummarizationMiddleware(
    model="gpt-4o-mini",
    max_tokens_before_summary=500,  # Trigger at 500 tokens
    messages_to_keep=5,  # Keep last 5 messages after summarizing
    summary_prompt="Summarize the conversation concisely, keeping key details."
)

summarizing_agent = create_agent(
    model="gpt-4o-mini",
    tools=[search_tool],
    middleware=[summarizer],
    system_prompt="You are a helpful assistant."
)

print("✓ Agent with automatic summarization created")
print("  - Summarizes when conversation exceeds 500 tokens")
print("  - Keeps last 5 messages")

✓ Agent with automatic summarization created
  - Summarizes when conversation exceeds 500 tokens
  - Keeps last 5 messages


### 1.4 Human-in-the-Loop Middleware

Pauses execution to get human approval before taking actions.

In [6]:
@tool
def send_email(to: str, subject: str, body: str) -> str:
    """Sends an email (simulated)."""
    return f"Email sent to {to} with subject '{subject}'"

# Human approval for sensitive actions
hitl_middleware = HumanInTheLoopMiddleware(
    interrupt_on={
        "send_email": {
            "allowed_decisions": ["approve", "reject", "edit"]
        }
    },
    description_prefix="⚠️ Action requires approval:"
)

# Need a checkpointer for HITL
checkpointer = InMemorySaver()

hitl_agent = create_agent(
    model="gpt-4o-mini",
    tools=[send_email],
    middleware=[hitl_middleware],
    checkpointer=checkpointer,
    system_prompt="You can send emails, but wait for approval."
)

print("✓ Agent with human-in-the-loop created")
print("  - Pauses before sending emails")
print("  - Requires explicit approval")

✓ Agent with human-in-the-loop created
  - Pauses before sending emails
  - Requires explicit approval


### 1.5 Todo List Middleware

Tracks tasks and checks them off as the agent completes them.

In [7]:
# Automatically tracks agent's plan
todo_middleware = TodoListMiddleware()

planning_agent = create_agent(
    model="gpt-4o-mini",
    tools=[search_tool],
    middleware=[todo_middleware],
    system_prompt="You are a research assistant who plans before acting."
)

print("✓ Agent with todo tracking created")
print("  - Automatically tracks planned tasks")
print("  - Updates as tasks complete")

✓ Agent with todo tracking created
  - Automatically tracks planned tasks
  - Updates as tasks complete


## Part 2: Decorator-Based Custom Middleware

Create custom middleware using decorators for quick, function-based hooks.

### 2.1 Input Validation Middleware

In [8]:
@before_model(can_jump_to=["end"])
def validate_input(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    """Check if the user's input is appropriate."""
    user_message = state['messages'][-1].content
    
    # Simple profanity check (in production, use a real filter)
    banned_words = ["badword1", "badword2"]
    
    for word in banned_words:
        if word.lower() in user_message.lower():
            logger.warning(f"🚫 Inappropriate content detected: {word}")
            return {
                "messages": [AIMessage("I cannot respond to inappropriate requests.")],
                "jump_to": "end"
            }
    
    logger.info("✅ Input validation passed")
    return None  # Continue normally

print("✓ Input validation middleware defined")

✓ Input validation middleware defined


### 2.2 Output Filtering Middleware

In [9]:
@after_model
def filter_output(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    """Remove sensitive information from the response."""
    last_message = state['messages'][-1]
    
    if hasattr(last_message, 'content') and last_message.content:
        content = last_message.content
        
        # Redact email addresses (simple regex)
        import re
        email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
        if re.search(email_pattern, content):
            filtered_content = re.sub(email_pattern, '[EMAIL_REDACTED]', content)
            logger.info("📧 Email addresses redacted")
            
            # Replace the message
            messages = state['messages'][:-1] + [AIMessage(filtered_content)]
            return {"messages": messages}
    
    return None  # No changes needed

print("✓ Output filtering middleware defined")

✓ Output filtering middleware defined


### 2.3 Logging Middleware

In [10]:
@before_agent
def log_start(state: AgentState, runtime: Runtime) -> None:
    """Log when agent execution starts."""
    user_msg = state['messages'][-1].content if state['messages'] else "No message"
    logger.info(f"🚀 Agent started | User: {user_msg[:50]}...")

@after_agent
def log_end(state: AgentState, runtime: Runtime) -> None:
    """Log when agent execution completes."""
    logger.info(f"✅ Agent completed | Total messages: {len(state['messages'])}")

print("✓ Logging middleware defined")

✓ Logging middleware defined


### 2.4 Create Agent with Custom Decorators

In [11]:
# Agent with all custom decorator middleware
custom_agent = create_agent(
    model="gpt-4o-mini",
    tools=[search_tool],
    middleware=[
        validate_input,
        filter_output,
        log_start,
        log_end
    ],
    system_prompt="You are a helpful assistant."
)

print("✓ Agent with custom decorator middleware created")
print("  - Input validation")
print("  - Output filtering (PII redaction)")
print("  - Start/end logging")

✓ Agent with custom decorator middleware created
  - Input validation
  - Output filtering (PII redaction)
  - Start/end logging


In [12]:
# Test the custom agent
result = custom_agent.invoke({
    "messages": [{"role": "user", "content": "Tell me about AI safety."}]
})

print("\nResponse:", result["messages"][-1].content[:200] + "...")

INFO:__main__:🚀 Agent started | User: Tell me about AI safety....
INFO:__main__:✅ Input validation passed
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:✅ Agent completed | Total messages: 2



Response: AI safety refers to the field of research and practice focused on ensuring that artificial intelligence systems operate safely and align with human values. As AI technology advances rapidly, concerns ...


## Part 3: Class-Based Custom Middleware

For more complex middleware with state, extend `AgentMiddleware` class.

### 3.1 Tool Monitoring Middleware

In [13]:
class ToolMonitoringMiddleware(AgentMiddleware):
    """Monitors and logs all tool executions."""
    
    def __init__(self):
        super().__init__()
        self.tool_calls = []
    
    def wrap_tool_call(
        self,
        request: ToolCallRequest,
        handler: Callable[[ToolCallRequest], ToolMessage],
    ) -> ToolMessage:
        """Intercept every tool call."""
        tool_name = request.tool_call['name']
        tool_args = request.tool_call['args']
        
        logger.info(f"🔧 Tool called: {tool_name}")
        logger.info(f"   Arguments: {tool_args}")
        
        try:
            # Execute the tool
            result = handler(request)
            
            # Track success
            self.tool_calls.append({
                "tool": tool_name,
                "args": tool_args,
                "status": "success"
            })
            
            logger.info(f"✅ Tool completed successfully")
            return result
            
        except Exception as e:
            # Track failure
            self.tool_calls.append({
                "tool": tool_name,
                "args": tool_args,
                "status": "error",
                "error": str(e)
            })
            
            logger.error(f"❌ Tool failed: {e}")
            raise
    
    def get_stats(self):
        """Get statistics about tool usage."""
        total = len(self.tool_calls)
        successful = sum(1 for call in self.tool_calls if call['status'] == 'success')
        failed = total - successful
        
        return {
            "total_calls": total,
            "successful": successful,
            "failed": failed,
            "calls": self.tool_calls
        }

print("✓ ToolMonitoringMiddleware class defined")

✓ ToolMonitoringMiddleware class defined


### 3.2 Performance Tracking Middleware

In [14]:
import time
from langchain.agents.middleware import ModelRequest, ModelResponse

class PerformanceMiddleware(AgentMiddleware):
    """Tracks execution time and token usage."""
    
    def __init__(self):
        super().__init__()
        self.model_calls = []
    
    def wrap_model_call(
        self,
        request: ModelRequest,
        handler: Callable[[ModelRequest], ModelResponse]
    ) -> ModelResponse:
        """Time each model call."""
        start_time = time.time()
        
        logger.info(f"⏱️  Model call starting...")
        
        # Execute the model call
        response = handler(request)
        
        # Calculate duration
        duration = time.time() - start_time
        
        # Track the call
        self.model_calls.append({
            "duration_seconds": duration,
            "timestamp": time.time()
        })
        
        logger.info(f"✅ Model call completed in {duration:.2f}s")
        
        return response
    
    def get_stats(self):
        """Get performance statistics."""
        if not self.model_calls:
            return {"total_calls": 0, "total_time": 0, "avg_time": 0}
        
        total_time = sum(call['duration_seconds'] for call in self.model_calls)
        avg_time = total_time / len(self.model_calls)
        
        return {
            "total_calls": len(self.model_calls),
            "total_time": total_time,
            "avg_time": avg_time,
            "calls": self.model_calls
        }

print("✓ PerformanceMiddleware class defined")

✓ PerformanceMiddleware class defined


### 3.3 Create Agent with Class-Based Middleware

In [15]:
# Initialize middleware instances
tool_monitor = ToolMonitoringMiddleware()
perf_tracker = PerformanceMiddleware()

# Create agent
monitored_agent = create_agent(
    model="gpt-4o-mini",
    tools=[search_tool],
    middleware=[tool_monitor, perf_tracker],
    system_prompt="You are a research assistant."
)

print("✓ Agent with monitoring middleware created")

✓ Agent with monitoring middleware created


In [16]:
# Test the monitored agent
result = monitored_agent.invoke({
    "messages": [{"role": "user", "content": "What is the latest news about quantum computing?"}]
})

print("\n" + "=" * 80)
print("RESPONSE:")
print("=" * 80)
print(result["messages"][-1].content[:300] + "...")

INFO:__main__:⏱️  Model call starting...
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:✅ Model call completed in 3.89s
INFO:__main__:🔧 Tool called: tavily_search_results_json
INFO:__main__:   Arguments: {'query': 'latest news about quantum computing'}
INFO:__main__:✅ Tool completed successfully
INFO:__main__:⏱️  Model call starting...
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:✅ Model call completed in 5.75s



RESPONSE:
Here are the latest developments in quantum computing:

1. **Breakthroughs and Real-World Applications**: A recent article discusses a significant shift in the quantum computing field towards practical implementations rather than merely focusing on processor benchmarks. This transition is expected t...


In [17]:
# View monitoring statistics
print("\n" + "=" * 80)
print("TOOL MONITORING STATS:")
print("=" * 80)
tool_stats = tool_monitor.get_stats()
print(f"Total tool calls: {tool_stats['total_calls']}")
print(f"Successful: {tool_stats['successful']}")
print(f"Failed: {tool_stats['failed']}")

print("\n" + "=" * 80)
print("PERFORMANCE STATS:")
print("=" * 80)
perf_stats = perf_tracker.get_stats()
print(f"Total model calls: {perf_stats['total_calls']}")
print(f"Total time: {perf_stats['total_time']:.2f}s")
print(f"Average time: {perf_stats['avg_time']:.2f}s")


TOOL MONITORING STATS:
Total tool calls: 1
Successful: 1
Failed: 0

PERFORMANCE STATS:
Total model calls: 2
Total time: 9.65s
Average time: 4.82s


## Part 4: Combining Multiple Middleware Layers

The real power comes from stacking multiple middleware together.

### 4.1 Production-Ready Agent with Full Stack

In [20]:
# Initialize all middleware
model_limiter = ModelCallLimitMiddleware(run_limit=5, exit_behavior="end")
tool_limiter = ToolCallLimitMiddleware(tool_name="tavily_search_results_json", run_limit=3)
summarizer = SummarizationMiddleware(
    model="gpt-4o-mini",
    max_tokens_before_summary=1000,
    messages_to_keep=5
)
tool_monitor = ToolMonitoringMiddleware()
perf_tracker = PerformanceMiddleware()

# Create production agent with full middleware stack
production_agent = create_agent(
    model="gpt-4o-mini",
    tools=[search_tool],
    middleware=[
        validate_input,      # Security first
        model_limiter,       # Cost control
        tool_limiter,        # Tool usage control
        summarizer,          # Context management
        tool_monitor,        # Observability
        perf_tracker,        # Performance tracking
        filter_output,       # Output security
        log_start,           # Logging
        log_end              # Logging
    ],
    system_prompt="You are a production research assistant with full monitoring."
)

print("✓ Production-ready agent created with full middleware stack:")
print("  1. Input validation")
print("  2. Model call limits")
print("  3. Tool call limits")
print("  4. Automatic summarization")
print("  5. Tool monitoring")
print("  6. Performance tracking")
print("  7. Output filtering")
print("  8. Comprehensive logging")

✓ Production-ready agent created with full middleware stack:
  1. Input validation
  2. Model call limits
  3. Tool call limits
  4. Automatic summarization
  5. Tool monitoring
  6. Performance tracking
  7. Output filtering
  8. Comprehensive logging


### 4.2 Test Production Agent

In [21]:
# Test with a complex query
result = production_agent.invoke({
    "messages": [{"role": "user", "content": "Research the latest breakthroughs in renewable energy."}]
})

print("\n" + "=" * 80)
print("PRODUCTION AGENT RESPONSE:")
print("=" * 80)
print(result["messages"][-1].content)

INFO:__main__:🚀 Agent started | User: Research the latest breakthroughs in renewable ene...
INFO:__main__:✅ Input validation passed
INFO:__main__:⏱️  Model call starting...
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:✅ Model call completed in 0.83s
INFO:__main__:🔧 Tool called: tavily_search_results_json
INFO:__main__:   Arguments: {'query': 'latest breakthroughs in renewable energy 2023'}
INFO:__main__:✅ Tool completed successfully
INFO:__main__:✅ Input validation passed
INFO:__main__:⏱️  Model call starting...
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:✅ Model call completed in 1.82s
INFO:__main__:🔧 Tool called: tavily_search_results_json
INFO:__main__:🔧 Tool called: tavily_search_results_json
INFO:__main__:   Arguments: {'query': 'current breakthroughs in solar energy 2023'}
INFO:__main__:   Arguments: {'query': 'current breakthroughs in wind energy 2023'}
INFO:_


PRODUCTION AGENT RESPONSE:
Here are some of the latest breakthroughs and developments in renewable energy from 2023:

### 1. Solar Energy
- **Perovskite Solar Cells**: Perovskite cells are emerging as a transformative technology in solar energy. These ultra-thin cells can be sprayed onto surfaces like windows and combined with silicon to enhance their energy absorption capabilities. Recent advancements include tandem solar cells reaching efficiencies of over 26.8%, indicating significant improvements in solar panel technology.
    - Article: [Perovskite: The 'wonder material' that could transform solar](https://www.bbc.com/future/article/20251015-perovskite-the-wonder-material-that-could-transform-solar-energy)
    - Additional trend: In early 2025, Trina Solar achieved a world record for solar conversion efficiency of 25.44% with n-type fully passivated heterojunction (HJT) solar modules, showcasing the continual improvement and competition within solar technology.
    - Source: [7 N

## Part 5: Middleware Execution Order

**Important**: Middleware execution order matters!

### Understanding Middleware Flow

```
REQUEST FLOW (Top to Bottom):
─────────────────────────────
User Input
    ↓
@before_agent    (ascending order)
    ↓
@before_model    (ascending order)
    ↓
🤖 LLM CALL
    ↓
@after_model     (descending order) ⚠️
    ↓
Tool Execution (if needed)
    ↓
@after_agent     (descending order) ⚠️
    ↓
Response to User
```

**Key Points:**
- `before_*` hooks execute in **ascending order** (first to last)
- `after_*` hooks execute in **descending order** (last to first)
- Plan your middleware stack accordingly!

## Part 6: Real-World Use Cases

### Use Case 1: Financial Analysis Agent with Compliance

In [22]:
@tool
def get_stock_price(symbol: str) -> str:
    """Get current stock price (simulated)."""
    return f"Stock {symbol}: $150.25"

@before_model(can_jump_to=["end"])
def check_compliance(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    """Ensure queries comply with financial regulations."""
    user_msg = state['messages'][-1].content.lower()
    
    # Check for insider trading language
    if "insider" in user_msg or "secret" in user_msg:
        logger.warning("⚠️ Compliance violation detected")
        return {
            "messages": [AIMessage("I cannot provide advice that may involve insider information.")],
            "jump_to": "end"
        }
    return None

# Financial agent with compliance
financial_agent = create_agent(
    model="gpt-4o-mini",
    tools=[get_stock_price],
    middleware=[
        check_compliance,
        ModelCallLimitMiddleware(run_limit=3, exit_behavior="end"),
        ToolMonitoringMiddleware()
    ],
    system_prompt="You are a financial analysis assistant. Always comply with regulations."
)

print("✓ Financial agent with compliance checks created")

✓ Financial agent with compliance checks created


### Use Case 2: Customer Support Agent with Escalation

In [23]:
@after_model
def detect_escalation(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    """Detect if issue needs escalation to human."""
    last_msg = state['messages'][-1].content if state['messages'] else ""
    
    # Check for escalation keywords
    escalation_keywords = ["angry", "frustrated", "manager", "lawsuit", "complaint"]
    
    user_messages = [msg.content for msg in state['messages'] if hasattr(msg, 'content')]
    combined = " ".join(user_messages).lower()
    
    for keyword in escalation_keywords:
        if keyword in combined:
            logger.warning(f"🚨 Escalation needed: '{keyword}' detected")
            escalation_msg = (
                "\n\n[SYSTEM NOTICE: This conversation has been flagged for human review "
                "due to complexity or customer sentiment.]"
            )
            updated_content = last_msg + escalation_msg
            messages = state['messages'][:-1] + [AIMessage(updated_content)]
            return {"messages": messages}
    
    return None

support_agent = create_agent(
    model="gpt-4o-mini",
    tools=[],
    middleware=[detect_escalation],
    system_prompt="You are a customer support agent. Be empathetic and helpful."
)

print("✓ Customer support agent with escalation detection created")

✓ Customer support agent with escalation detection created


## Key Takeaways

### What We Learned
✅ **Built-in middleware**: Ready-to-use for common patterns  
✅ **Decorator-based**: Quick custom hooks with `@before_model` etc.  
✅ **Class-based**: Complex middleware with state  
✅ **Composability**: Stack multiple layers for production  
✅ **Execution order**: Critical for proper behavior  

### Middleware Types

| Type | Use Case | Complexity |
|------|----------|------------|
| Built-in | Standard patterns | Low |
| Decorator | Quick custom hooks | Medium |
| Class-based | Complex with state | High |

### Best Practices
1. **Layer logically**: Security → Limits → Processing → Output
2. **Test individually**: Verify each middleware works alone
3. **Monitor performance**: Track overhead from middleware
4. **Keep it focused**: Each middleware should have one job
5. **Document order**: Be explicit about execution sequence

### Common Patterns
- **Security Stack**: Input validation → Content filter → Output sanitization
- **Cost Control**: Model limits → Tool limits → Summarization
- **Observability**: Logging → Monitoring → Performance tracking
- **Compliance**: Regulatory checks → Audit logging → Approval gates

### Next Steps
1. **Implement guardrails**: Add safety layers for production
2. **Create middleware library**: Build reusable components
3. **Add telemetry**: Integrate with monitoring systems
4. **Test edge cases**: Ensure middleware handles errors
5. **Optimize performance**: Profile and reduce overhead

## Resources

- [LangChain 1.0 Middleware Guide](https://docs.langchain.com/oss/python/langchain/middleware)
- [Middleware API Reference](https://reference.langchain.com/python/langchain/middleware/)
- [Context Engineering Blog](https://blog.langchain.com/agent-middleware/)
- [LangChain GitHub](https://github.com/langchain-ai/langchain)